This notebook analyzes a strategy for inivesting in an index fund based on the level of correction in the stock market. The strategy is based on the idea that the stock market is cyclical and that the market will eventually recover from a correction. The strategy is to invest in an index fund when the market is down by a certain percentage.

In this particular strategy, we will:
- Invest INR 50,000/- in an index fund when the market is down by 1% to 1.5%.
- nvest INR 100,000/- in an index fund when the market is down by 1.5% or more.

In [3]:
import yfinance as yf
import pandas as pd

In [4]:
SHORT_TERM_CAPITAL_GAINS_TAX_RATE = 0.20
BROKERAGE = 0
SECURITY_TRANSACTION_TAX_RATE = 0.001
TRANASCTION_CHARGES_RATE = 0.0000297
GST_RATE = 0.18
SEBI_CHARGES = 10
STAMP_DUTY_RATE = 0.00015 # Applicable only in case of buy transaction

def calculate_STCG(price, quantity, buy_price, sell_price):
    """
    Calculate the Short Term Capital Gains Tax
    """
    return max((sell_price - buy_price), 0) * quantity * SHORT_TERM_CAPITAL_GAINS_TAX_RATE

def calculate_charges(price, quantity, buy=True):
    """
    Calculate the charges for a transaction
    """
    if buy:
        stamp_duty = price * quantity * STAMP_DUTY_RATE
    else:
        stamp_duty = 0
    brokerage = BROKERAGE
    stt = price * quantity * SECURITY_TRANSACTION_TAX_RATE
    transaction_charges = price * quantity * TRANASCTION_CHARGES_RATE
    gst = (brokerage + SEBI_CHARGES + transaction_charges) * GST_RATE
    return brokerage + stt + transaction_charges + gst + SEBI_CHARGES + stamp_duty

In [5]:
# Define the ticker symbol for NIFTY 50
# ticker = 'NIFTYBEES.NS' # NIFTY 50 ETF
# ticker = 'NIF100BEES.NS' # NIFTY 100 ETF
ticker = 'NV20BEES.NS'

# Set the time period
start_date = "2013-01-01"
end_date = "2025-02-28"

# Fetch the data
data = yf.download(ticker, start=start_date, end=end_date, interval='1d')
data.index = data.index.date
pd.DataFrame.drop(data, columns=['Close', 'Volume'], inplace=True)
data.rename(columns={'Adj Close': 'Close'}, inplace=True)

data['Returns'] = data['Close'].pct_change() + 1
data.dropna(inplace=True)

CURRENT_PRICE = data['Close'].iloc[-1]

[*********************100%%**********************]  1 of 1 completed


In [6]:
AMOUNT_TO_INVEST = 0.0

for index, row in data.iterrows():
    AMOUNT_TO_INVEST = 100000 if row['Returns'] < 0.985 else 50000 if row['Returns'] < 0.99 else 0
    quantity = AMOUNT_TO_INVEST // row['Close']
    data.at[index, 'Quantity'] = quantity
    data.at[index, 'BUY_Cost'] = row['Close'] * quantity
    data.at[index, 'Profit'] = ((CURRENT_PRICE - row['Close']) * quantity)
    data.at[index, 'CurrentValue'] = data.at[index, 'BUY_Cost'] + data.at[index, 'Profit']

In [7]:
data

,Open,High,Low,Close,Returns,Quantity,BUY_Cost,Profit,CurrentValue
2018-05-18,49.400002,49.715000,49.400002,49.715000,0.988370,1005.0,49963.575153,99449.773006,149413.34816
2018-06-01,49.564999,49.564999,49.564999,49.564999,0.996983,0.0,0.000000,0.000000,0.00000
2018-06-07,50.430000,50.436001,50.152000,50.436001,1.017573,0.0,0.000000,0.000000,0.00000
2018-06-14,50.810001,50.814999,50.539001,50.701000,1.005254,0.0,0.000000,0.000000,0.00000
2018-06-22,54.200001,54.200001,50.200001,51.090000,1.007672,0.0,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...
2025-02-05,156.399994,156.399994,152.119995,152.589996,1.001904,0.0,0.000000,-0.000000,0.00000
2025-02-06,155.639999,155.639999,151.429993,152.220001,0.997575,0.0,0.000000,-0.000000,0.00000
2025-02-07,156.800003,156.800003,150.309998,150.880005,0.991197,0.0,0.000000,-0.000000,0.00000
2025-02-10,150.889999,152.669998,149.410004,150.050003,0.994499,0.0,0.000000,-0.000000,0.00000


In [8]:
import plotly.graph_objects as go
import pandas as pd

fig = go.Figure()

# Plot the 'Close' data
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close', line=dict(color='blue')))

# Plot the 'BUY_Cost' between 49000 and 51000
buy_cost_49_51 = data[(data['BUY_Cost'] > 49000) & (data['BUY_Cost'] < 51000)]
fig.add_trace(go.Scatter(x=buy_cost_49_51.index, y=buy_cost_49_51['Close'], mode='markers', name='1-1.5% drop', marker=dict(color='green', symbol='star')))

# Plot the 'BUY_Cost' greater than 99000
buy_cost_99 = data[data['BUY_Cost'] > 99000]
fig.add_trace(go.Scatter(x=buy_cost_99.index, y=buy_cost_99['Close'], mode='markers', name='>1.5% drop', marker=dict(color='red', symbol='circle')))

# Update layout for better visualization
fig.update_layout(
    # title='Interactive Plot with Hover Data',
    xaxis_title='Date', yaxis_title='Close',
    hovermode='x',
    width=1600, height=1200,
    legend=dict(
        x=0.01, y=0.99,
        traceorder='normal', bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='Black',
        borderwidth=1
    )
)

# Show the plot
fig.show()

In [9]:
from finlib import calculate_cagr, format_inr

total_investment = data['BUY_Cost'].sum()
nett_profit = data['Profit'].sum()
cagr = calculate_cagr(data['BUY_Cost'], data.index, data['CurrentValue'].sum())

print(f'Total Investment: {format_inr(total_investment)}')
print(f'Nett Profit: {format_inr(nett_profit)}')
print(f'CAGR: {cagr:.2%}')

Total Investment: ₹1,26,42,829.57
Nett Profit: ₹1,18,95,599.63
CAGR: 21.59%
